# AI问答---产品技术文档和问题集
> **作者：** lsc
> 
> **时间：** 20231126
>
> **程序工作简介：** 选择多个本地文件，用本地聊天窗口调用chatdocAPI进行对话，并不断更新聊天内容给对话窗口和api
> 
> **关注的点：** 星火文档问答评估下来实际能实际应用呢？和chatgpt对比如何呢？能加入一些提示词结果能好点吗？评估测试方式怎么样还能再完善吗？

**背景介绍&意义：** 
   - langchain刚出的时候用chatgpt知识库问答，原理和能做的事情的边界就不赘述，当时体会下来就是token数量有限，理解能力有限，无法实际应用；时隔半年多，由厂家来实现的效果怎么样呢；
   - 在用户真实使用过程中，大家根据现象用自己的语言去描述，大家表述都不同，所以"模糊的或换了个说法的问题能回答好吗"
   - 在真实的文档中，存在结构化的文字，比如表格等等，所以“大模型能理解并作出正确回答吗”
   - 作为一个电子工程师，开发完一个产品，有交付给PCB工程师控制器能力限制说明文档，和给到测试和工程现场的使用人员的异常处理问题集

**参与对象：**
   1. 星火大模型 文档问答api，https://chatdoc.xfyun.cn/
   2. 作为对比项，大模型直接测试：gpt3.5 、 gpt4.0

**评估方式：**
> 之前测试过langchain+chatgpt，大模型的优势很清楚，测试简单问题虽能升任，但是实际场景复杂;
> 在用户真实使用过程中，用户会根据现象用自己的语言去描述，大家表述都不同，很可能大模型就开始胡说八道了，而不是像人类一样不懂就是不懂，然后会有一个清晰的思路一步步追问
   1. 上传
      1. 《***控制卡的带载能力说明》
   2. 问答类型（模糊的或换了个说法的问题，理解表格等结构化的文字的能力）
      1. 之前测试过langchain+chatgpt，这里就不测试简单问题了，在用户真实使用过程中，大家根据现象用自己的语言去描述，大家表述都不同
      2. 

**结果**
   1. xfyun-chatdoc比不过gpt4，gpt3.5，
   2. 但是比星火大模型强，星火大模型输入一个文档后，并没有采用文档内容去做专业回答，哪怕我做了提示（希望是个bug）
   3. gpt4在模糊回答上，联想能力准确；但是对md表格理解不了,表格如下


         | 变量A | 变量B | 变量C       |
      | -------- | ------------ | ------------------ |
      | <= 32    | <= 80        | <= 128             |
      | <= 32    | <= 160       | <= 64              |
      | ...    | ...       | ...            |
   4. xfyun-chatdoc在一些模糊回答上，就只能做到早期langchain的水平，把上下文相关的整理一下给你列出来回答
**大模型现阶段缺陷**
   1. 不能自己建立逻辑链条，在用户问不到点子上的时候，只有答案正确或错误，并不能一步步走向正确
      - 在用户真实使用过程中，用户会根据现象用自己的语言去描述，大家表述都不同，很可能大模型就开始胡说八道了，而不是像人类一样不懂就是不懂，然后会有一个清晰的思路一步步追问【可以通过promt来实现几个场景的思路提示，但无法泛化的模型，可能还不如原来的笨办法】
   2. 不能有效理解结构化语言，比如markdown表格
   3. 现有大模型多模态功能还不够强大，无法推理出图片中存在什么异常现象【这个可能就得自己训练专有模型才行了】
      - 比如一个显示屏异常，但是用户拍摄时当时拍摄的视频画面，角度各不相同，此时无法推理出图片中存在什么异常现象

## 内容介绍
1. 准备工作：
   1. 工具：anaconda + python + jupyter
   2. 申请资料：讯飞问答api接口https://chatdoc.xfyun.cn/

In [6]:
# 讯飞问答api接口https://chatdoc.xfyun.cn/，需要自己申请appid和appsecret
APPId = "dd5b8534"
APISecret = "OTMxNmE4MmFlNTMzNmEyMjBhYzUyY2E2"

2. 程序工作步骤：
   1. 填写api所需id和secretkey
   2. 选择本地文件
   3. 进行聊天问答



# 程序正文
## 选择本地多个文件

In [7]:
import tkinter as tk
from tkinter import filedialog

file_paths = []  # 存储选择的文件路径

def open_file_dialog():
    global file_paths
    file_paths = filedialog.askopenfilenames(
        title="选择文件",
        filetypes=[("所有文件", "*.*")],
        initialdir="/",  # 设置初始目录，可根据需要修改
        multiple=True  # 允许选择多个文件
    )
    process_files_and_close()

def process_files_and_close():
    # 打印文件名
    for file_path in file_paths:
        print("文件名:", file_path)
    
    # 关闭窗口
    root.destroy()

# 创建主窗口
root = tk.Tk()
root.title("文件选择示例")

# 设置窗口的初始宽度和高度
root.geometry("400x200")  # 按需调整宽度和高度

# 添加按钮
button = tk.Button(root, text="选择文件", command=open_file_dialog)
button.pack(pady=20)

# 运行主循环
root.mainloop()


文件名: O:/WorkFile/xinghuo-chatdoc-api-python-demo/QA_data/全彩控制卡的带载能力说明 - 副本.txt


## 上传

In [8]:
# -*- coding:utf-8 -*-
import hashlib
import base64
import hmac
import time
import random
from urllib.parse import urlencode
import json
import requests
from requests_toolbelt.multipart.encoder import MultipartEncoder
class Document_Upload:
    def __init__(self, APPId, APISecret, timestamp):
        self.APPId = APPId
        self.APISecret = APISecret
        self.Timestamp = timestamp

  
    def get_origin_signature(self):
        m2 = hashlib.md5()
        data = bytes(self.APPId + self.Timestamp, encoding="utf-8")
        m2.update(data)
        checkSum = m2.hexdigest() 
        return checkSum


    def get_signature(self):
        # 获取原始签名
        signature_origin = self.get_origin_signature()
        # 使用加密键加密文本
        signature = hmac.new(self.APISecret.encode('utf-8'), signature_origin.encode('utf-8'),
                                 digestmod=hashlib.sha1).digest()
        # base64密文编码
        signature = base64.b64encode(signature).decode(encoding='utf-8')
        return signature

    def get_header(self):
        signature = self.get_signature()
        header = {
            "appId": self.APPId,
            "timestamp": self.Timestamp,
            "signature": signature,
        }
        return header

    # 提交网络文件
    def get_body(self):
        body = {
            "file": "",
            "url": "http://xingh.dsd.xss/e-0867bed7d1a3/c5a58172-8c1c-4c79-90e0-de4042ba876d1691741866951.txt",
            "fileName": "c5a58112313876d1691741866951.txt",
            "fileType": "wiki",
            "callbackUrl": "your_callbackUrl"
        }
        form = MultipartEncoder(
            fields=body,
            boundary='------------------' + str(random.randint(1e28, 1e29 - 1))
        )
        return form

    # 提交本地文件
    def get_files_and_body(self, filepath):
        files = {'file': open(filepath, 'rb')} 
        #通过Windows视窗的方式，让用户选择文件，存放到files内
        # files = {'file1': open(tkinter.filedialog.askopenfilename(), 'rb')}
        body = {
            "url": "",
            "fileName": filepath,
            "fileType": "wiki",
            "needSummary": False,
            "stepByStep": False,
            "callbackUrl": "your_callbackUrl",
        }
        return files, body

fileIds = []
if __name__ == '__main__':
    curTime = str(int(time.time()))
    request_url = "https://chatdoc.xfyun.cn/openapi/fileUpload"

    

    # ******************提交网络文件
    # body = document_upload.get_body()
    # headers['Content-Type'] = body.content_type
    # response = requests.post(request_url, data=body, headers=headers)

    # ******************提交本地文件
    for file_path in file_paths:
        document_upload = Document_Upload(APPId, APISecret, curTime)
        headers = document_upload.get_header()
        files, body = document_upload.get_files_and_body(file_path)
        response = requests.post(request_url, files=files, data=body, headers=headers)
        fileId = json.loads(response.text)['data']['fileId']
        fileIds.append(fileId)
        print('\nonMessage:\n' + json.dumps(json.loads(response.text), indent=4, ensure_ascii=False))

    #打印fileIds列表中的内容
    print(fileIds)


    # # print("\n请求头:\n", response.request.headers, type(response.request.headers))
    # # json格式化说出请求头
    # print('\n请求头:\n' + json.dumps(dict(response.request.headers), indent=4, ensure_ascii=False))

    # print('\nonMessage:\n' + json.dumps(json.loads(response.text), indent=4, ensure_ascii=False))
    # # response.text转换成json，然后输出其中data.fileId的值
    # print('\nfileId:\n' + json.loads(response.text)['data']['fileId'])

    # 文档上传成功



onMessage:
{
    "flag": true,
    "code": 0,
    "desc": null,
    "data": {
        "parseType": "TEXT",
        "quantity": "3",
        "fileId": "a2729621641c4ded904512205945667b"
    },
    "sid": "8a00cd846f4e4e64aab4ae29db8d4fd1"
}
['a2729621641c4ded904512205945667b']


## 回答

In [9]:
# -*- coding:utf-8 -*-
import hashlib
import base64
import hmac
import time
from urllib.parse import urlencode
import json
import websocket
import _thread as thread
import ssl


class Document_Q_And_A:
    def __init__(self, APPId, APISecret, TimeStamp, OriginUrl ):
        self.appId = APPId
        self.apiSecret = APISecret
        self.timeStamp = TimeStamp
        self.originUrl = OriginUrl

    def get_origin_signature(self):
        m2 = hashlib.md5()
        data = bytes(self.appId + self.timeStamp, encoding="utf-8")
        m2.update(data)
        checkSum = m2.hexdigest()
        return checkSum



    def get_signature(self):
        # 获取原始签名
        signature_origin = self.get_origin_signature()
        # print(signature_origin)
        # 使用加密键加密文本
        signature = hmac.new(self.apiSecret.encode('utf-8'), signature_origin.encode('utf-8'),
                             digestmod=hashlib.sha1).digest()
        # base64密文编码
        signature = base64.b64encode(signature).decode(encoding='utf-8')
        # print(signature)
        return signature



    def get_header(self):
        signature = self.get_signature()
        header = {
            "Content-Type": "application/json",
            "appId": self.appId,
            "timestamp": self.timeStamp,
            "signature": signature
        }
        return header

    def get_url(self):
        signature = self.get_signature()
        header = {
            "appId": self.appId,
            "timestamp": self.timeStamp,
            "signature": signature
        }
        return self.originUrl + "?" + f'appId={self.appId}&timestamp={self.timeStamp}&signature={signature}'
        # 使用urlencode会导致签名乱码
        # return self.originUrl + "?" + urlencode(header)



    def get_body(self , messages , fileIds):
        data = {
            "chatExtends": {
                "wikiPromptTpl": "请将以下内容作为已知信息：\n<wikicontent>\n请根据以上内容回答用户的问题。\n问题:<wikiquestion>\n回答:",
                "wikiFilterScore": 65,
                "temperature": 0.5,
                "sparkWhenWithoutEmbedding": 1
            },
            "fileIds": fileIds,
            "messages": messages
        }
        return data

# 收到websocket错误的处理
def on_error(ws, error):
    print("### error:", error)


# 收到websocket关闭的处理
def on_close(ws, close_status_code, close_msg):
    print("### closed ###")
    print("关闭代码：", close_status_code)
    print("关闭原因：", close_msg)


# 收到websocket连接建立的处理
def on_open(ws):
    thread.start_new_thread(run, (ws,))


def run(ws, *args):
    data = json.dumps(ws.question)
    ws.send(data)

received_messages = []
received_status = None
# 收到websocket消息的处理
def on_message(ws, message):
    global received_messages, received_status
    data = json.loads(message)
    code = data['code']
    if code != 0:
        print(f'请求错误: {code}, {data}')
        ws.close()
    else:
        received_messages.append(data["content"])
        received_status = data["status"]
        print(data["content"], end='')
        if received_status == 2:
            ws.close()


def Q_and_A(messages_gbl , fileIds_gbl):
    global received_messages, received_status
    # Reset received_messages and received_status at the start
    received_messages = []
    received_status = None

    curTime = str(int(time.time()))
    # curTime = "1690945325"
    OriginUrl = "wss://chatdoc.xfyun.cn/openapi/chat"
    document_Q_And_A = Document_Q_And_A(APPId, APISecret, curTime, OriginUrl )

    wsUrl = document_Q_And_A.get_url()
    print(wsUrl)
    headers = document_Q_And_A.get_header()
    body = document_Q_And_A.get_body(messages_gbl, fileIds_gbl)

    # 禁用WebSocket库的跟踪功能，使其不再输出详细的调试信息。
    websocket.enableTrace(False)
    ws = websocket.WebSocketApp(wsUrl, on_message=on_message, on_error=on_error, on_close=on_close, on_open=on_open)
    ws.appid = APPId
    ws.question = body
    ws.run_forever(sslopt={"cert_reqs": ssl.CERT_NONE})

    # Wait for status to be 2
    while received_status != 2:
        time.sleep(0.1)  # Avoid busy waiting

    # Combine all received messages into a single string
    combined_content = ''.join(received_messages)
    return combined_content

## 交互页面

In [10]:
import tkinter as tk

# Import other necessary libraries and functions here

class ChatApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Chat Application")
        
        # 设置行列权重以使它们可以伸缩
        root.grid_rowconfigure(0, weight=1)
        root.grid_columnconfigure(0, weight=1)

        # 创建文本框来显示消息，设置为自适应伸缩
        self.messages_text = tk.Text(root, wrap=tk.WORD)
        self.messages_text.grid(row=0, column=0, columnspan=2, sticky="nsew")

        # 创建输入框，设置为只在水平方向伸缩
        self.input_entry = tk.Entry(root)
        self.input_entry.grid(row=1, column=0, sticky="ew")

        # 创建发送按钮
        self.send_button = tk.Button(root, text="Send", command=self.send_user_input)
        self.send_button.grid(row=1, column=1, sticky="ew")

        # Initialize a list to store messages
        self.messages = []

    def send_user_input(self):
        # Get the user's input from the entry widget
        user_input = self.input_entry.get()

        # Add user's question to the messages list
        self.messages.append({"role": "user", "content": user_input})

        # Clear the -User input field
        self.input_entry.delete(0, tk.END)

        # Call Q_and_A function to get the assistant's response
        assistant_response = self.get_assistant_response()

        # Add assistant's response to the messages list
        self.messages.append({"role": "assistant", "content": assistant_response})
        # print(self.messages)
        # Update the messages text widget
        self.update_messages_text()

    def get_assistant_response(self):
        # Implement the logic to call the Q_and_A function with user_question
        # and retrieve the assistant's response
        # You should call your Q_and_A function here and return the response
        # For now, you can return a placeholder response
        return Q_and_A(self.messages , fileIds )

    def update_messages_text(self):
        # Clear the current content of the messages text widget
        self.messages_text.delete(1.0, tk.END)

        # Iterate through the messages and insert them into the text widget
        for message in self.messages:
            role = message["role"]
            content = message["content"]
            self.messages_text.insert(tk.END, f"{role}: {content}\n")

if __name__ == "__main__":
    root = tk.Tk()
    app = ChatApp(root)
    root.mainloop()


wss://chatdoc.xfyun.cn/openapi/chat?appId=dd5b8534&timestamp=1701015973&signature=IJp9O/XGSRBCDvvOTJcUtpFmPIU=
根据给出的信息，8个端口，每个端口宽度为155，高度为200，满足带载要求。因为对于PORT个数 <= 32的情况，显示PORT的宽 <= 80，高 <= 128，而这里每个端口的宽度和高度都小于这个限制。

# 结果
## 结果1
> 不理想
>

```
user: 1
assistant: Hello! How can I assist you today?
user: 单组数据线带载的显示区域限制条件
assistant: 单组数据线带载的显示区域限制条件如下：

简单情况下，每组RGB带载范围及走线完全相同，此时一组显示数据线带载的区域就是一个显示PORT的大小。

复杂情况下，每组RGB带载范围或走线不同，但相邻两组RGB组合成PORT，PORT间的带载和走线情况完全相同。此时，两组相邻的数据线带载的区域是一个显示PORT的大小。

在NPWM程序中，显示PORT的宽 > 10；在PWM程序中，无限制。
user: 当port个数为32，显示port宽为80，显示port的高最大为多少
assistant: 当port个数为32，显示port宽为80时，显示port的高最大为128。
user: 当port个数为32，显示port宽为160，显示port的高最大为多少
assistant: 当port个数为32，显示port宽为160时，显示port的高最大为64。
user: 重复单元内的IC数量最多是多少
assistant: 重复单元内的IC数量最多是16个。
user: 什么情况下IC数量可以为32
assistant: 在复杂情况下，重复单元内的IC数量可以为32。
user: 什么情况下IC数量可以为32
assistant: 在复杂情况下，重复单元内的IC数量可以为32。
user: 在复杂情况下，重复单元内的IC数量可以为
assistant: 在复杂情况下，重复单元内的IC数量可以为16。

```

### 存在的问题
1. 不能理顺因果关系，不理解文章结构的内涵，只会语义检索

## 结果2

```
user: 你好
assistant: 你好！有什么我可以帮助你的吗？
user: pwm芯片拍照效果不好怎么办
assistant: pwm芯片拍照效果不好的原因可能有以下几点：

1. 低灰均匀性改小。可以尝试调整低灰均匀性参数，使其更接近正常值。
2. 缩小同步时间。将同步时间从1开始往上增加，直到显示正常（CG01/CG02-3.082以上，CG03-1.007以上支持同步时间缩小到15us以下）。
3. 提高刷新率或改为3D模式。可以尝试提高刷新率或改为3D模式，以提高画面质量。

如果以上方法仍无法解决问题，建议联系电子工程师进行进一步排查和处理。
user: 5124芯片，如何调节dck周期的宽度
assistant: 要调节5124芯片的dck周期宽度，可以按照以下步骤操作：

1. 调低同步时间和换行时间。
2. 调节灰度排布表，高扫常用时的dck宽度 > 均衡 > 低扫常用；如果还不行，联系电子工程师制作特殊的自定义灰度排布表。
3. 如何查看dck宽度是多少，在导出文件夹的info文件内。

具体操作方法如下：

1. 打开控制板软件，选择“新建方案”。
2. 在“基本设置”中，找到“第一扫偏暗补偿时间”，将其数值调低，然后点击“确定”。
3. 在“高级模式”中，找到寄存器0x0D和0x0F，将它们的数值也调低，然后点击“确定”。
4. 如果调整后仍然无法达到理想的亮度效果，可以尝试重新选择刷新率，然后重复上述步骤。

```

### 存在的问题
1. 不能理解前后因果关系
    - 原文内容
        ```
        ## npwm芯片如何调节dck周期的宽度

        - 调低同步时间和换行时间
        - 调节灰度排布表，高扫常用时的dck宽度 > 均衡 > 低扫常用；如果还不行，联系电子工程师制作特殊的自定义灰度排布表
        - 如何查看dck宽度是多少，在导出文件夹的info文件内
        ```
    - 存在的问题
        下面这部分内容不是正确答案，但是因为属于问题的上文内容，所以给回答出来了

        ```
        具体操作方法如下：

        1. 打开控制板软件，选择“新建方案”。
        2. 在“基本设置”中，找到“第一扫偏暗补偿时间”，将其数值调低，然后点击“确定”。
        3. 在“高级模式”中，找到寄存器0x0D和0x0F，将它们的数值也调低，然后点击“确定”。
        4. 如果调整后仍然无法达到理想的亮度效果，可以尝试重新选择刷新率，然后重复上述步骤。
        ```
    - 好的地方
       - 我问的是5124，没有问npwm，还是找出了位置

# 大模型直接测试
> 惊讶于GPT4,我提供的文档里面没有提到“全彩LED”这几个字，他直接说出来这是“这些信息似乎是关于全彩LED控制卡的技术文档”
## 问题
记住我发的内容，根据内容回答问题,《全彩控制卡的带载能力说明.md》
## ChatGPT
- ChatGPT

    ```
    我已经记住了您提供的内容，请问您有什么问题需要我回答或者解释吗？
    ```
- User
`什么情况下IC数量可以为32`
- ChatGPT


    ```
    根据您提供的信息，IC数量可以为32的情况是在以下条件下：

    在简单情况下，每组RGB带载范围及走线完全相同，并且符合以下条件之一：

    数据线带载的区域是一个显示PORT的大小，且显示PORT的高不超过128像素。
    数据线带载的区域是一个显示PORT的大小，且显示PORT的宽不超过80像素。
    在复杂情况下，两组相邻的数据线带载的区域是一个显示PORT的大小，且显示PORT的高不超过128像素。

    请注意，这些条件是为了确保在数据线带载的情况下，每组数据线上的IC数量可以达到32。在其他情况下，IC数量可能会受到限制，并不一定可以达到32。
    ```
## gpt4
- ChatGPT

    ```
    根据您提供的内容，这些信息似乎是关于全彩LED控制卡的技术文档。它们详细地描述了控制卡的各种技术规格和限制条件。请告诉我您具体想了解哪部分内容，或者有什么具体问题需要解答？这样我可以更准确地根据您提供的内容来回答。
    ```
- User
`什么情况下IC数量可以为32`
- ChatGPT

    ```
    在您提供的文档中，IC数量能达到32的情况出现在“单组数据线带载的IC数量限制”部分。根据这部分的内容，IC数量可达到32的条件分为两个情况：

    简单情况下：这是指当重复单元内的每组RGB带载范围及走线完全相同时，重复单元内的IC数量可以达到32。

    复杂情况下：这通常指的是当每组RGB带载范围或走线不同，但相邻两组RGB组合成一个PORT，并且PORT间的带载和走线情况完全相同时。在这种情况下，重复单元内的IC数量通常限制为16。

    因此，只有在简单情况下，即每组RGB带载范围及走线完全相同的情况下，单组数据线带载的IC数量才能达到32。
    ```
## 星火
- xinghuo

    ```
    根据提供的文档，以下是回答你的问题：
    。。。总结了一下文档
    ```
- User
`什么情况下IC数量可以为32`
- xinghuo

    ```
    在连接"标准逻辑IC"时，需要考虑一个输出最大可连接的IC数量。这个数量受到输出电流的限制，我们把允许连接的IC上限个数称为扇出。例如在TTL IC中，只要考虑到TTL IC是由双极性晶体管构成的，就能容易地理解开关切换时是需要电流的。因此，如果一个IC的输出电流可以满足连接32个其他IC的需求，那么就可以连接32个IC。同时，也要注意所连接的设备是否在同一电压等级，以及电路设计的合理性和稳定性。
    ```